In [7]:
import random
from langchain_core.tools import tool

@tool("random_number")
def random_number_maker(input: str):
    """Returns a random number between 0-1000."""
    return random.randint(0, 1000)

random_number_maker.run('')

829

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") # "ised-openai-gpt4"
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

print(AZURE_OPENAI_DEPLOYMENT_NAME)
# print(AZURE_OPENAI_API_KEY)
# print(AZURE_ENDPOINT)
# print(AZURE_OPENAI_VERSION)

# model_folder = "/home/ubuntu/models/GGUF/"
# model_name = "mistral-7b-instruct-v0.1.Q8_0.gguf"
# model_path = model_folder + model_name

# n_gpu_layers = 42  # Change this value based on your model and your GPU VRAM pool.
# n_batch = 1024  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# n_ctx = 4096

ised-openai-gpt4


In [9]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.llms import LlamaCpp

# llm = LlamaCpp(
#     model_path=model_path,
#     temperature=0,
#     max_tokens=4096,
#     top_p=1,
#     n_gpu_layers=n_gpu_layers,
#     n_batch=n_batch,
#     n_ctx= n_ctx
# )

llm = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    azure_endpoint=AZURE_ENDPOINT,
    temperature=0,
    streaming=True
    # callbacks=[StreamingStdOutCallbackHandler()]
)

llm.bind_tools([random_number_maker])

RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x75b3507759f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x75b350777130>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', streaming=True, azure_endpoint='https://ised-openai-model.openai.azure.com/openai/deployments/ised-openai-gpt4/chat/completions?api-version=2023-07-01-preview', deployment_name='ised-openai-gpt4', openai_api_version='2023-07-01-preview', openai_api_type='azure'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'random_number', 'description': 'Returns a random number between 0-1000.', 'parameters': {'type': 'object', 'properties': {'input': {'type': 'string'}}, 'required': ['input']}}}]})

## Test LangGraph

In [11]:
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage

graph_init = MessageGraph()

graph_init.add_node("oracle", llm)

graph_init.add_edge("oracle", END)

graph_init.set_entry_point("oracle")

runnable = graph_init.compile()

runnable.invoke(HumanMessage(content="What is the remainder from 139 and 70?"))

[HumanMessage(content='What is the remainder from 139 and 70?', id='c2051f07-7291-4804-b525-17d5c2f288c2'),
 AIMessage(content='The remainder when you divide 139 by 70 is 69.', response_metadata={'finish_reason': 'stop'}, id='run-ac756cec-1e5f-4762-9386-d67b2546cda8-0')]